Lab 10
---
Hello All,

Welcome to the DSCI 510's lab.

Guidelines for the Lab:
- You will be given the lab assignment in the start of the lab. You're supposed to complete it by the deadline stated on blackboard.  

- You've to complete the assignments individually. If you are having trouble completing the assignment do let me know, I will clear your doubts and guide you but I'll not write code for you and no one else should :) !!!  

- You have to fill in the code in this notebook and upload it to back to blackboard for submission. While doing this, make sure that all supporting files that you download from blackboard are in the same directory as this notebook.  

- You are encouraged to look up resources online like python docs and stackoverflow. But, you are encouraged to look up the topics and not the questions themselves  

- Your last submission will be counted towards your grade  

Before attempting the below question, I recommend you to skim this xml documentation: https://docs.python.org/3/library/xml.etree.elementtree.html

Q1.[30 points] 
---
You are given a xml file with cities. See the attached file for its format. Takeaway from the format is that there is a data root tag which has child city tags in it. Each city tag may have multiple attributes and multiple tags in it. These tags and attributes are not mandated to be there in each city tag.  

For this question, you are given a xml_file path and a `filters` dictionary as arguments of a function `filter_cities`. The key of the `filters` dictionary represents the attribute name of the city tag or the child tag name of the `city` tag. The value of that key represents the value of the corresponding attribute or text of the corresponding tag name.  

This function should return the list of name of the cities that have matching values for all the attributes and tags given in the `filters`.  

Assumptions: 
- All the attribute and tag names are combinedly unique.
- All cities will have `name` tag in it. 

Note: While extracting the text from a tag, use `strip` to remove empty spaces and new lines.

---
Grading Rubric:  
Using the sample xml file only.  
- +10 - If function outputs correctly for `filters` that has only attribute name keys in it.  
- +10 - If function outputs correctly for `filters` that has only tag name keys in it.  

Using any kind of xml file with the above mentioned structure.  
- +10 - If function outputs correctly for `filters` for all cases.  

In [6]:
import xml.etree.ElementTree as ET
# Your code goes here
def filter_cities(xml_file, filters):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    
    keys = [i for i in filters]
    attribs = list(set([i for city in root.findall(root[0].tag) for i in city.attrib.keys()]))
    lst_total = []
    for key in keys:
        if key in attribs:
            lst = []
            for city in root.findall(root[0].tag):
                try:
                    if city.attrib[key]==filters[key]:
                        lst.append(city.find('name').text.strip())
                except:
                    continue
            lst_total.append(lst)
        else:
            lst = []
            for city in root.findall(root[0].tag):
                try:
                    if city.find(key).text.strip()==filters[key]:
                        lst.append(city.find('name').text.strip())
                except:
                    continue
            lst_total.append(lst)
    #common_city = list(set(lst_total[0]).intersection(*lst_total[1:]))
    result = set(lst_total[0])
    for i in lst_total[1:]:
        result &= set(i)
    common_city = list(result)
         
    return common_city

In [7]:
filters = {
    "class": "A",
    "country": "USA"
}
print(filter_cities("city.xml", filters))
# Expected output: ['Los Angeles', 'New York']

['Los Angeles', 'New York']


---

In [48]:
#intersection

#1 reduce
from functools import reduce
L = [[1,2,3,4], [2,3,4,5], [3,4,5,6],[1,2,3,4,5,6]]
reduce(lambda x,y : set(x) & set(y), L)

#2 set
L = [[1,2,3,4], [2,3,4,5], [3,4,5,6],[1,2,3,4,5,6]]
set(L[0]).intersection(*L[1:])   # '*' means to break up list as separate arguments

#3 set loop
L = [[1,2,3,4], [2,3,4,5], [3,4,5,6],[1,2,3,4,5,6]]
result = set(L[0])
for i in L[1:]:
    result &= set(i)
result

In [28]:
#using lxml and creating xpath
from lxml import etree
def filter_cities(xml_file, filters):
    with open(xml_file) as f:
        tree = etree.parse(f)
        
    # construct xpath
    #string = ''
    #for i in filters:
     #   if i=='class':
      #      string += ' and @class={}'.format("'"+filters[i]+"'")
       # else:
        #    string += ' and contains({},{})'.format(i,"'"+filters[i]+"'")
    #string = string.strip().lstrip('and ')
    
    lst_xpath = []
    for i in filters:
        if i=='class':
            lst_xpath.append('@class={}'.format("'"+filters[i]+"'"))
        else:
            lst_xpath.append('contains({},{})'.format(i,"'"+filters[i]+"'"))
    string = ' and '.join(lst_xpath)
    
    xpath = '//city[' + string + ']/name/text()'
    
    lst = [i.strip() for i in tree.xpath(xpath)]
    return lst

filters = {
    "class": "A",
    "country": "USA"
    #"state"
    #"name"
    #"county"
}
print(filter_cities("city.xml", filters))

['Los Angeles', 'New York']


Bonus Question.[5 points] 
---
To complete this question, you need the access key from the opencage website. To obtain that the procedure is:  
1. open https://opencagedata.com/api
2. Click on sign up for your free api key.
3. Create a account and use the access key that you get from there for this question.

You are given a xml_file and `filters` dictionary(Same as the above question(Also in the same format)). You have to get the latitude and longitude for the cities that you returned in the above question. Use the opencagedata api to do geocoding. Give the city name as the input and you will get a json response with lat and long information in it.   

Input: str, dict  
Return Output: List of tuples(each element in format (lat, long))  

Note: Make use of the above question's function. If you cannot complete the above question, you can assume that it exists. 

---
Grading Rubric: Binary

In [75]:
import json
import requests

def filter_cities_with_latlong(xml_file, filters):
    lst = filter_cities(xml_file, filters)
    lst_loc = []
    for i in lst:
        url = 'https://api.opencagedata.com/geocode/v1/json?key=2d8c00546c334deaaed81a61f798baf4&q={}&pretty=1'.format(i)
        req = requests.get(url)
        jsn = req.text
        dic = json.loads(jsn)
        longitude = dic['results'][0]['geometry']['lng']
        latitude = dic['results'][0]['geometry']['lat']
        lst_loc.append((latitude,longitude))
    return lst_loc

filters = {
    "class": "A",
    "country": "USA"
}
filter_cities_with_latlong("city.xml", filters)

[(34.0536909, -118.242766), (40.7127281, -74.0060152)]

In [18]:
#solution2 using OpenCageGeocode

from opencage.geocoder import OpenCageGeocode

def filter_cities_with_latlong(xml_file, filters):
    key = '2d8c00546c334deaaed81a61f798baf4'
    geocoder = OpenCageGeocode(key)
    try: 
        lst = filter_cities(xml_file, filters)
        lst_loc = []
        for i in lst:
            results = geocoder.geocode(i, no_annotations='1')           
            if results and len(results):
                longitude = results[0]['geometry']['lng']
                latitude  = results[0]['geometry']['lat']
                #print(u'%f;%f;%s' % (latitude, longitude, i))
                # 34.053691;-118.242766;Los Angeles
                # 40.712728;-74.006015;New York
                lst_loc.append((latitude,longitude))
            else:
                sys.stderr.write("not found: %s\n" % address)
    except IOError:
        print('Error: File %s does not appear to exist.' % addressfile)
    return lst_loc   


filters = {
    "class": "A",
    "country": "USA"
}
filter_cities_with_latlong("city.xml", filters)

[(34.0536909, -118.242766), (40.7127281, -74.0060152)]